In [ ]:
# from google.colab import drive
# drive.mount("/content/drive")

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
import joblib


In [ ]:
dataset_df_1=pd.read_csv('/content/chungcu_batdongsan1900.csv')
#dataset_df_1=dataset_df_1.drop('date_submitted',axis=1)

dataset_df_2=pd.read_csv('/content/chungcu_batdongsan1900_3.csv')
#dataset_df_2=dataset_df_2.drop('date_submitted',axis=1)

dataset_df_3=pd.read_csv('/content/chungcu_batdongsan_4csv.csv')
dataset_df_3=dataset_df_3.drop('date_submitted',axis=1)

dataset_df_4=pd.read_csv('/content/chungcu_batdongsan_5csv.csv')
dataset_df_4=dataset_df_4.drop('date_submitted',axis=1)

dataset_df_5=pd.read_csv('/content/chungcu_batdongsan1900_6.csv')
dataset_df_5=dataset_df_5.drop('date_submitted',axis=1)



In [ ]:
dataset_df = pd.concat([dataset_df_1, dataset_df_2,dataset_df_3,dataset_df_4,dataset_df_5], ignore_index=True)


In [ ]:
dataset_df_4.head()

,title,links,price,area,price_per_m2,bedroom,toilet,location,description,balcony_direction,house_direction,property_legal_document,furniture
0,"Cần bán gấp CH chung cư Kingdom, Q10, 72m2, 2P...",https://batdongsan.com.vn/ban-can-ho-chung-cu-...,"4,9 tỷ",72 m²,"~68,06 triệu/m²",2 phòng,2 phòng,"Dự án Kingdom 101, Đường Tô Hiến Thành, Phường...","Cần bán gấp căn hộ chung cư kingdom, Quận 10. ...",0,0,0,Căn bản
1,Cần bán căn hộ Ecolife _2PN_2WC_64m2 căn mới 1...,https://batdongsan.com.vn/ban-can-ho-chung-cu-...,"1,08 tỷ",64 m²,"~16,88 triệu/m²",2 phòng,2 phòng,"Dự án Ecolife Riverside, Thành phố Quy Nhơn, B...",Dành cho a/c nào cần mua Ecolife để ở hoặc đầu...,Tây - Nam,Tây - Nam,Sổ đỏ/ Sổ hồng,0
2,"Chính chủ cần bán căn 4PN, DT 132m2, tòa S3, v...",https://batdongsan.com.vn/ban-can-ho-chung-cu-...,12 tỷ,132 m²,"~90,91 triệu/m²",4 phòng,3 phòng,"Dự án Vinhomes Skylake, Đường Phạm Hùng, Phườn...","Cần bán căn hộ 3PN, tòa S3, view hồ điều hòa. ...",Đông - Bắc,Tây - Nam,Sổ đỏ/ Sổ hồng,0
3,"Cần bán 2,05 tỷ - 102% (59m2 - 2PN + 2WC) - tầ...",https://batdongsan.com.vn/ban-can-ho-chung-cu-...,"2,05 tỷ",59 m²,"~34,75 triệu/m²",2 phòng,2 phòng,"Dự án Westgate, Đường Tân Túc, Thị trấn Tân Tú...",Chính chủ cần bán căn hộ 2PN - 2WC.\n+ Giá HD:...,0,0,Sổ đỏ/ Sổ hồng,Không nội thất
4,"Cần bán căn hộ chung cư Hà Đô Centrosa, Quận 1...",https://batdongsan.com.vn/ban-can-ho-chung-cu-...,"6,5 tỷ",86 m²,"~75,58 triệu/m²",2 phòng,2 phòng,"Dự án HaDo Centrosa Garden, Đường 3/2, Phường ...","Cần bán gấp căn hộ chung cư Hà Đô Centrosa, Qu...",0,0,0,0


In [ ]:
# Data clean
dataset_df['area'] = dataset_df['area'].replace('[^\d.]', '', regex=True).astype(float)
dataset_df['price_per_m2'].replace(',', '.', regex=True, inplace=True)
dataset_df['price_per_m2'] = dataset_df['price_per_m2'].replace('[^\d.]', '', regex=True).astype(float)


In [ ]:

dataset_df['price_per_m2'] = dataset_df['price_per_m2'].astype(float)

dataset_df['price_per_m2'] = dataset_df['price_per_m2'].astype(float)

dataset_df = dataset_df[dataset_df['location'] != 0]


dataset_df['bedroom'] = dataset_df['bedroom'].replace('[^\d.]', '', regex=True).astype(float)
dataset_df['toilet'] = dataset_df['toilet'].replace('[^\d.]', '', regex=True).astype(float)

dataset_df['balcony_direction'] = dataset_df['balcony_direction'].replace('0', pd.NA, regex=False)
dataset_df['house_direction'] = dataset_df['house_direction'].replace('0', pd.NA, regex=False)
dataset_df['furniture'] = dataset_df['furniture'].replace('0', pd.NA, regex=False)

dataset_df = dataset_df.dropna(subset=['balcony_direction'])
dataset_df = dataset_df.dropna(subset=['house_direction'])
dataset_df = dataset_df.dropna(subset=['furniture'])

dataset_df['location']=dataset_df['location'].apply(lambda x: x.split(",") )
dataset_df= dataset_df.reset_index(drop=True)


In [ ]:
dataset_df

,title,links,price,area,price_per_m2,bedroom,toilet,location,description,balcony_direction,house_direction,property_legal_document,furniture
0,Booking TC3 - 100 căn đầu tiên CK khủng giá từ...,https://batdongsan.com.vn/ban-can-ho-chung-cu-...,Thỏa thuận,62.0,0.00,2.0,2.0,[Dự án The Canopy Residences - Vinhomes Smart ...,- The Canopy có 2 hầm để xe và cạnh nhà 2 để x...,Tây - Bắc,Đông - Nam,Sổ đỏ/ Sổ hồng,Cơ bản
1,Chỉ từ 700 triệu sở hữu căn hộ sông Hàn CĐT Su...,https://batdongsan.com.vn/ban-can-ho-chung-cu-...,"1,6 tỷ",27.0,59.26,1.0,1.0,"[Dự án Sun Ponte Residence Đà Nẵng, Đường Trầ...",Căn hộ ngắm cầu Rồng pháo hoa mặt sông Hàn tru...,Đông - Nam,Đông - Nam,Sổ đỏ/ Sổ hồng,Đầy đủ
2,Độc quyền! Quỹ căn 3N nhỉnh 3tỷ 98m2 Grand Sun...,https://batdongsan.com.vn/ban-can-ho-chung-cu-...,"3,19 tỷ",98.0,32.55,3.0,2.0,"[Dự án Grand Sunlake, Đường Trần Phú, Phường...",Grand Sunlake - căn hộ chung cư hot nhất Hà Đô...,Đông - Nam,Đông - Nam,Hợp đồng mua bán,Đầy đủ
3,Không báo giá ảo! Quỹ căn mới nhất 4/2024 CC c...,https://batdongsan.com.vn/ban-can-ho-chung-cu-...,"3,7 tỷ",9166.0,40.41,3.0,2.0,"[Dự án Grand Sunlake, Đường Trần Phú, Phường...",ĐỘC QUYỀN! Quỹ căn hộ cao cấp Grand Sunlake VI...,Đông - Nam,Đông - Nam,Sổ đỏ/ Sổ hồng,Đầy đủ
4,Rổ hàng ngộp cắt lỗ sâu Vinhomes Q9 - Studio 1...,https://batdongsan.com.vn/ban-can-ho-chung-cu-...,"1,2 tỷ",30.0,40.00,1.0,1.0,"[Dự án Vinhomes Grand Park, Phường Long Thạnh...","5SHOMES. VN xin chào quý anh/chị.\nHiện tại, B...",Đông,Tây - Nam,Sổ đỏ/ Sổ hồng,Đầy đủ
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7517,"C?n bán can CC 2PN - 2VS, 73m2 CC New Life t?n...",https://batdongsan.com.vn/ban-can-ho-chung-cu-...,"1,86 t?",73.0,25.48,2.0,2.0,"[D? án New Life Tower, Ðu?ng Hoàng Qu?c Vi?t,...","Giá 1,86 t? d? l?i h?t n?i th?t nhu hình,\nS? ...",Nam,B?c,S? d?/ S? h?ng.,Ð?y d?.
7518,C?t l? c?c sâu can À La Carte H? Long view chí...,https://batdongsan.com.vn/ban-can-ho-chung-cu-...,"45,5 tri?u/m²",45.0,2.05,1.0,1.0,"[D? án A La Carte H? Long Bay, Phu?ng Hùng Th...","A La Carte giá c?t l? ch? còn 2,1t? so v?i h?p...",Tây - Nam,Ðông - B?c,S? d?/ S? h?ng,Ð?y d?
7519,S?c! giá r? gi?t mình! gd c?n tr? bank g?p tro...,https://batdongsan.com.vn/ban-can-ho-chung-cu-...,"4,9 t?",902.0,54.32,3.0,2.0,"[D? án The Tonkin - Vinhomes Smart City, Phu?...",GÐ C?n bán g?p can 3PN thi?t k? d?p giá th?p h...,Ðông - Nam,Ðông - B?c,H?p d?ng mua bán,Ð?y d?.
7520,Bán can 3PN 2VS giá ch? 850tr can CC Dragon c?...,https://batdongsan.com.vn/ban-can-ho-chung-cu-...,Th?a thu?n,93.0,0.00,3.0,2.0,"[D? án The Dragon Castle, Ðu?ng Cái Lân, Phu...",3PN giá 850tr t?ng ngay 135tr n?i th?t ngay hô...,Tây - B?c,Ðông - Nam,H?p d?ng mua bán,Co b?n


In [ ]:
for index, x in enumerate(dataset_df['location']):
    if len(x) < 5:
        print("Error in row {}: {}".format(index, x))
        dataset_df=dataset_df.drop(index,axis=0)





Error in row 0: ['Dự án The Canopy Residences - Vinhomes Smart City', ' Phường Tây Mỗ', ' Nam Từ Liêm', ' Hà Nội']
Error in row 4: ['Dự án Vinhomes Grand Park', ' Phường Long Thạnh Mỹ', ' Quận 9', ' Hồ Chí Minh']
Error in row 5: ['Dự án The Miami', ' Phường Tây Mỗ', ' Nam Từ Liêm', ' Hà Nội.']
Error in row 6: ['Dự án Vinhomes Golden River Ba Son', ' Quận 1', ' Hồ Chí Minh']
Error in row 8: ['Dự án Vinhomes Grand Park', ' Phường Long Thạnh Mỹ', ' Quận 9', ' Hồ Chí Minh']
Error in row 11: ['Dự án The Canopy Residences - Vinhomes Smart City', ' Phường Tây Mỗ', ' Nam Từ Liêm', ' Hà Nội.']
Error in row 12: ['Dự án Vinhomes Ocean Park Gia Lâm', ' Xã Dương Xá', ' Gia Lâm', ' Hà Nội']
Error in row 17: ['Dự án The Miami', ' Phường Tây Mỗ', ' Nam Từ Liêm', ' Hà Nội']
Error in row 18: ['Dự án Vinhomes Grand Park', ' Phường Long Thạnh Mỹ', ' Quận 9', ' Hồ Chí Minh']
Error in row 19: ['Dự án The Zurich - Vinhomes Ocean Park', ' Xã Đa Tốn', ' Gia Lâm', ' Hà Nội']
Error in row 26: ['Dự án The Miami',

In [ ]:
dataset_df['Tỉnh']=dataset_df['location'].apply(lambda x :x[-1] )
dataset_df['Quận']=dataset_df['location'].apply(lambda x :x[-2] )
dataset_df['Phường']=dataset_df['location'].apply(lambda x :x[-3] )
dataset_df['Đường']=dataset_df['location'].apply(lambda x :x[-4] )


In [ ]:
dataset_df = dataset_df[dataset_df['price_per_m2'] != 0]


In [ ]:
dataset_df=dataset_df[(dataset_df['Tỉnh'] ==' Hà Nội')|(dataset_df['Tỉnh'] ==' Hồ Chí Minh') ]
dataset_df.head()

,title,links,price,area,price_per_m2,bedroom,toilet,location,description,balcony_direction,house_direction,property_legal_document,furniture,Tỉnh,Quận,Phường,Đường
2,Độc quyền! Quỹ căn 3N nhỉnh 3tỷ 98m2 Grand Sun...,https://batdongsan.com.vn/ban-can-ho-chung-cu-...,"3,19 tỷ",98.0,32.55,3.0,2.0,"[Dự án Grand Sunlake, Đường Trần Phú, Phường...",Grand Sunlake - căn hộ chung cư hot nhất Hà Đô...,Đông - Nam,Đông - Nam,Hợp đồng mua bán,Đầy đủ,Hà Nội,Hà Đông,Phường Văn Quán,Đường Trần Phú
3,Không báo giá ảo! Quỹ căn mới nhất 4/2024 CC c...,https://batdongsan.com.vn/ban-can-ho-chung-cu-...,"3,7 tỷ",9166.0,40.41,3.0,2.0,"[Dự án Grand Sunlake, Đường Trần Phú, Phường...",ĐỘC QUYỀN! Quỹ căn hộ cao cấp Grand Sunlake VI...,Đông - Nam,Đông - Nam,Sổ đỏ/ Sổ hồng,Đầy đủ,Hà Nội,Hà Đông,Phường Văn Quán,Đường Trần Phú
7,Quỹ căn bán chuyển nhượng tháng 4/2024 giá tốt...,https://batdongsan.com.vn/ban-can-ho-chung-cu-...,"5,8 tỷ",80.0,72.50,2.0,2.0,"[Dự án Times City, Đường Minh Khai, Phường V...",Vinhomes Times City - Park Hill xin cảm ơn quý...,Nam,Bắc,Sổ đỏ/ Sổ hồng,Cơ bản,Hà Nội,Hai Bà Trưng,Phường Vĩnh Tuy,Đường Minh Khai
9,"Qũy căn bán mới giá tốt chỉ từ hơn 1,5 tỷ chun...",https://batdongsan.com.vn/ban-can-ho-chung-cu-...,"2,59 tỷ",51.0,50.86,1.0,1.0,"[Dự án Masteri Water Front, Đường Quốc Lộ 5...",* Masteri WaterFront là dự án chung cư cao cấp...,Đông - Nam,Tây - Bắc,Hợp đồng mua bán.,Tiêu chuẩn bàn giao cao cấp nhất nhất Vinhomes...,Hà Nội,Gia Lâm,Xã Đa Tốn,Đường Quốc Lộ 5B
10,Tổng hợp quỹ căn 1 - 5 ngủ bán Times City - Pa...,https://batdongsan.com.vn/ban-can-ho-chung-cu-...,"4,5 tỷ",75.0,60.00,2.0,2.0,"[Dự án Times City, Đường Minh Khai, Phường V...",C&C Homes đơn vị chuyển nhượng và cho thuê số ...,Nam,Bắc,Sổ đỏ/ Sổ hồng,Đầy đủ,Hà Nội,Hai Bà Trưng,Phường Vĩnh Tuy,Đường Minh Khai


In [ ]:
len(dataset_df)

1352

In [ ]:


y = dataset_df['price_per_m2']
X = dataset_df.drop('price_per_m2', axis=1)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tạo các pipeline xử lý dữ liệu
numerical_cols = ['area', 'bedroom', 'toilet']  # thay thế các giá trị thực tế của bạn
categorical_cols = ['balcony_direction',	'house_direction',	'property_legal_document',	'furniture','Đường','Tỉnh',	'Quận',	'Phường']  # thay thế các giá trị thực tế của bạn

#
my_cols=categorical_cols+numerical_cols
X_train = X_train[my_cols]
X_test = X_test[my_cols]

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Định nghĩa lại my_pipeline với một RandomForestRegressor mặc định
my_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(random_state=42))
])

# Function to create the model with hyperparameter space
# Define the parameter distribution
param_dist = {
    'model__n_estimators': randint(200, 250),
  'model__max_depth': randint(4, 7),
    'model__min_samples_split': randint(2, 4)
}

# Create a RandomizedSearchCV object and fit it to the training data
random_search = RandomizedSearchCV(
    estimator=my_pipeline,
    param_distributions=param_dist,
    n_iter=100,
    cv=3,
    verbose=1,
    n_jobs=-1,
    random_state=42
)


In [ ]:
random_search.fit(X_train, y_train)

# Print the best parameters
print("Best parameters found: ", random_search.best_params_)

# Your best model is now fitted with the best set of hyperparameters
best_model = random_search.best_estimator_

# Use the best model to make predictions
val_preds = best_model.predict(X_test)
print("Validation MAE: ", mean_absolute_error(y_test, val_preds))

# Save the best model using joblib
#Best parameters found:  {'model__max_depth': 6, 'model__min_samples_split': 2, 'model__n_estimators': 221}


Fitting 3 folds for each of 100 candidates, totalling 300 fits
Best parameters found:  {'model__max_depth': 6, 'model__min_samples_split': 2, 'model__n_estimators': 232}
Validation MAE:  12.522460556546484


In [ ]:
joblib.dump(best_model, 'model_chung_cu.pkl')

['model_chung_cu.pkl']